In [19]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

In [20]:
import scipy.sparse as sparse

# create matrix A to apply forward difference scheme
def forward_diff_matrix(n):
    data = []
    i = []
    j = []
    for k in range(n - 1):
        i.append(k)
        j.append(k)
        data.append(-1)

        i.append(k)
        j.append(k+1)
        data.append(1)
        
    # incidence matrix of the 1-d mesh
    return sparse.coo_matrix((data, (i,j)), shape=(n-1, n)).tocsr()

def Laplacian(n):
    """
    Create Laplacian on 2-dimensional grid with n*n nodes
    """
    B = forward_diff_matrix(n)
    D = -B.T @ B
    Dx = sparse.kron(sparse.eye(n), D).tocsr()
    Dy = sparse.kron(D, sparse.eye(n)).tocsr()
    return Dx + Dy

In [22]:
n = 100
h=(1/n)**2
x = np.linspace(0,1,n)
L = Laplacian(n)
L = L.todense()*h

#print(np.shape(L))  #(40000, 40000)

In [2]:
from sir import *

In [3]:
def SIR_continuous2(b,p,k,time,ii,startpos,M):
    """
    Simulates continuous SIR model
    ii = initial percentage of infected
    time = Days of simulation
    b = probability that people getting infectious 
    k = probability that people getting recovered
    M = number of grid in each side
    
    returns sol from solve_ivp
    """
    pop = [Person(startpos) for i in range(N)]
    initial_infection = randint(N,size=np.int(N*ii))
    for i in initial_infection:
        pop[i].infection()
    S = np.zeros((M,M))
    I = np.zeros((M,M))
    R = np.zeros((M,M))
    l=1/M
    for i in range(N):
        index_x=np.floor(pop[i].pos/np.array([l,l]))[0]
        index_y=np.floor(pop[i].pos/np.array([l,l]))[1]
        if pop[i].is_susceptible:
            S[int(index_x),int(index_y)]+=1
        if pop[i].is_infected():
            I[int(index_x),int(index_y)]+=1
        if pop[i].is_removed():
            R[int(index_x),int(index_y)]+=1
    Sflat=S.flatten()/N
    Iflat=I.flatten()/N
    Rflat=R.flatten()/N
            
            
    def SIR(t, X):
        z=np.zeros((M*M))
        Y=np.append(np.append(z,z),z)
        Y[0:M*M] = -b * X[0:M*M] * X[2*M*M:] + p * L @ X[0:M*M]
        Y[M*M:2*M*M] = k * X[2*M*M:] + p * L @ X[M*M:2*M*M]
        Y[2*M*M:] = b * X[0:M*M] * X[2*M*M:] - (k * X[2*M*M:]) + p * L @ X[2*M*M:]
        return Y
    t_eval = np.linspace(0, time, 1000)
    y0=np.append(np.append(Sflat,Rflat),Iflat)
    sol1 = solve_ivp(SIR, [0, time], y0, method='RK45', t_eval=t_eval)    # solve the equation
    return sol1


In [ ]:
from sir import *

b=1
p=0.01
k=1/3
time=150
ii=0.01
startpos=[0.5,0.5]
M=200

sol1=SIR_continuous2(b,p,k,time,ii,startpos,M)

In [14]:
sol1.y

In [24]:
b=1
p=0.01
k=1/3
time=150
N=100
ii=0.01
M=200
startpos=[0.5,0.5]
pop = [Person(startpos) for i in range(N)]
initial_infection = randint(N,size=np.int(N*ii))
for i in initial_infection:
    pop[i].infection()
S = np.zeros((M,M))
I = np.zeros((M,M))
R = np.zeros((M,M))
l=1/M
for i in range(N):
    index_x=np.floor(pop[i].pos/np.array([l,l]))[0]
    index_y=np.floor(pop[i].pos/np.array([l,l]))[1]
    if pop[i].is_susceptible():
        S[int(index_x),int(index_y)]+=1
    if pop[i].is_infected():
        I[int(index_x),int(index_y)]+=1
    if pop[i].is_removed():
        R[int(index_x),int(index_y)]+=1
Sflat=S.flatten()/N
Iflat=I.flatten()/N
Rflat=R.flatten()/N
def SIR(t, X):
    z=np.zeros((M*M))
    Y=np.append(np.append(z,z),z)
    Y[0:M*M] = -b * X[0:M*M] * X[2*M*M:] + p * L @ X[0:M*M]
    Y[M*M:2*M*M] = k * X[2*M*M:] + p * L @ X[M*M:2*M*M]
    Y[2*M*M:] = b * X[0:M*M] * X[2*M*M:] - (k * X[2*M*M:]) + p * L @ X[2*M*M:]
    return Y
t_eval = np.linspace(0, time, 100)
y0=np.append(np.append(Sflat,Rflat),Iflat)
sol1 = solve_ivp(SIR, [0, time], y0, method='RK45', t_eval=t_eval)    # solve the equation


KeyboardInterrupt: 

In [53]:
a=np.array([0., 1., 2.])
b=np.array([0., 2., 4.])
t=np.append(np.append(a,b),b)
t[:2*2]

array([0., 1., 2., 0.])

In [31]:
np.floor(0.2)

0.0

In [43]:
N=1000
ii=0.01
l=1/200
pop = [Person() for i in range(N)]
initial_infection = randint(N,size=np.int(N*ii))
for i in initial_infection:
    pop[i].infection()
for i in range(N):
    j=np.floor(pop[i].pos/np.array([l,l]))[0]
    if j==200:
        print(i)
